# CIFAR-10


https://github.com/geifmany/cifar-vgg

In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.datasets import cifar10
from cifar10vgg import cifar10vgg
from time import time


from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing

from tqdm import tqdm

Using TensorFlow backend.


ModuleNotFoundError: No module named 'cifar10vgg'

In [ ]:
from sklearn.metrics import make_scorer, balanced_accuracy_score
from scipy.stats import pearsonr

def discrete_variable_score(y_true, y_predicted):    
    return balanced_accuracy_score(y_true, y_predicted)

def continuous_variable_score(y_true, y_predicted):
    return pearsonr(y_true,y_predicted)[0]


def extract_representations(model,samples):
    inp = model.input                                           # input placeholder
    outputs = [layer.output for layer in model.layers]          # all layer outputs
    functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function
    layers_outs = functor([samples, 1.])
    return layers_outs

# create scorer
my_func = make_scorer(discrete_variable_score,greater_is_better=True)

In [ ]:
def load_cifar10_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_test_int = y_test
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)    
    return x_train, x_test, y_train, y_test, y_test_int

In [ ]:
x_train, x_test, y_train, y_test, y_test_int = load_cifar10_data()

In [ ]:
model = cifar10vgg(train=False)

In [ ]:
model.model.summary()

In [ ]:
n_samples = 100
samples = x_test[:n_samples,:,:,:]
labels  = y_test[:n_samples,:]
labels_int = y_test_int[:n_samples]
print(samples.shape)
print(labels.shape)

In [ ]:
tin = time()
predicted_x = model.predict(samples)
print('Elapsed : {}'.format(time() - tin))

In [ ]:
residuals = np.argmax(predicted_x,1) != np.argmax(labels,1)
loss = sum(residuals)/len(residuals)
print("the validation 0/1 loss is: ",loss)

In [ ]:
np.argmax(predicted_x[:10],1)

In [ ]:
np.argmax(labels[:10],1)

In [ ]:
layers_outs = extract_representations(model.model, samples)
layers_names = [l.name for l in model.model.layers]

In [ ]:
_ = [print(i, name, l.shape) for i, (name, l) in enumerate(zip(layers_names, layers_outs) )]

In [ ]:
# patiently individuate dropout and pooling layers indexes 
idx = [3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 52, 57, 59]
selected_outs = [(layers_names[i],layers_outs[i]) for i in idx]

# or use a nice list comprehension
selected_outs = [(layers_names[i],layers_outs[i]) for i in range(len(layers_names)) 
                 if 'dropout' in layers_names[i] or 'pool' in layers_names[i]]

selected_layers_labels = [l for (l,_) in selected_outs]

# Decoding

In [ ]:
Score = []
Results = []
Parameters = []


Cs = np.power(10., np.arange(-3,3))
#Cs = [0.01]

# define grid of parameters
param_grid = {'C' : Cs}
nresampling = 10
nfolds = 5
nprocs = 1


y = labels_int.ravel()

for (name, representation) in tqdm(selected_outs):
    
    X = np.reshape(representation, (n_samples, -1))
    
    # preprocessing
    #X = preprocessing.scale(X) 
    
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=None)
    
    clf_ = SVC(kernel='linear')
    
    grid_search = GridSearchCV(clf_, param_grid, 
                                scoring=my_func, 
                                cv=nfolds, 
                                n_jobs=nprocs, 
                                refit=True,
                                return_train_score=True)

    grid_search.fit(X_train, y_train)    
    
    
    Score.append(grid_search.best_score_)             
    Results.append(grid_search.cv_results_)
    Parameters.append(grid_search.best_params_['C'])

In [ ]:
fs = 20
ms = 20

fig = plt.figure(figsize=(10,10))
plt.plot(range(len(selected_outs) ), Score, '-bo',markersize=ms) 
plt.xticks(range(len(selected_outs) ), 
           selected_layers_labels, 
           rotation='vertical',
           fontsize=fs)

plt.yticks(fontsize=fs)
plt.ylabel('balanced accuracy',fontsize=fs)
plt.xlabel('layer',fontsize=fs)
plt.show()